<a href="https://colab.research.google.com/github/gianmarco-holm/PY01_MachineLearning_LinearRegression_HousePrices/blob/main/PY01_MachineLearning_LinearRegression_HousePrices_V1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Regresión Lineal para Multiples Variable Dependiente
En esta notebook veremos como se entrena un modelo de Machine Learning con el arlgoritmo de regresión lineal para saber el precio de una casa a partir de dos variables independiente que es el número de habitaciones y el índice de industrialización alrededor de la casa.

## Importando datos

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data',
            header=None, sep='\s+')
df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


## Creando modelo con sklearn

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

X = df[['RM', 'INDUS']].values
y = df['MEDV'].values.reshape(-1, 1)
sc_x = StandardScaler()
sc_y = StandardScaler()
X_std = sc_x.fit_transform(X)
y_std = sc_y.fit_transform(y)

slr = LinearRegression()
slr.fit(X_std, y_std)

LinearRegression()

## Análisis de resultados

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [ ]:
# Crea un rango de valores desde el mínimo hasta el máximo
# Este rango se utilizará para crear la malla de la superficie.
# arange devuelve un array unidimensional
x1_range = np.arange(df['RM'].min(), df['RM'].max())
x2_range = np.arange(df['INDUS'].min(), df['INDUS'].max())

# Crea una cuadrícula de valores a partir de los rangos de 'RM' e 'INDUS'.
# X1 contiene las coordenadas de 'RM' y X2 las de 'INDUS' para cada punto en la cuadrícula.
# Esto devuelve una matriz o array bidimensional
X1, X2 = np.meshgrid(x1_range, x2_range)

# Convierte la cuadrícula de malla en un DataFrame para que pueda ser utilizada como entrada en el modelo de predicción.
# 'ravel()' aplana los arrays X1 y X2, esto significa que los vuelve en array unidimensional para formar el dataframe.
plano = pd.DataFrame({'RM': X1.ravel(), 'INDUS': X2.ravel()})

# Realiza predicciones utilizando el modelo de regresión lineal entrenado 'slr' en el DataFrame de la cuadrícula.
# La salida es un array unidimensional de predicciones, que se remodela para coincidir con la forma de X1 que es una matriz o array bidimensional.
pred = slr.predict(plano).reshape(X1.shape)

# Invierte la estandarización de las predicciones para obtener los valores originales de 'MEDV'.
pred = sc_y.inverse_transform(pred)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



In [ ]:
# Forma 1
# Crea una nueva figura para el gráfico.
fig = plt.figure()

# Añade un eje 3D a la figura para que podamos hacer un gráfico tridimensional.
ax = fig.add_subplot(111, projection='3d')  # Usa add_subplot para 3D

# Dibuja una superficie 3D utilizando las coordenadas de la malla (X1, X2) y las predicciones (pred).
# 'alpha=0.4' establece la transparencia de la superficie.
ax.plot_surface(X1, X2, pred, alpha=0.4)

# Añade un gráfico de dispersión 3D de los datos originales al gráfico.
# Utiliza 'RM' e 'INDUS' como coordenadas x e y, y 'MEDV' como coordenada z.
# 'color='red'' establece el color de los puntos en rojo y 'marker='o'' define el marcador como un punto.
ax.scatter3D(df['RM'], df['INDUS'], df['MEDV'], color='red', marker='o')

# Establece el ángulo de visión inicial del gráfico 3D.
# 'elev=10' establece la elevación y 'azim=5' establece el azimut.
ax.view_init(elev=10, azim=5)

# Etiquetas de los ejes
ax.set_xlabel('RM')
ax.set_ylabel('INDUS')
ax.set_zlabel('Predicción')

# Muestra el gráfico.
plt.show()

In [ ]:
# Forma 2

# LIBs
import plotly.express as px
import plotly.graph_objects as go

# Genera el gráfico 3D de dispersión utilizando plotly.express.
fig = px.scatter_3d(df, x='RM', y='INDUS', z='MEDV')

# Actualiza las trazas del gráfico para ajustar el tamaño del marcador.
fig.update_traces(marker=dict(size=5))

# Añade una superficie de predicción al gráfico 3D.
fig.add_traces(go.Surface(x=X1, y=X2, z=pred, name='pred_surface'))

# Muestra el gráfico.
fig.show()

## Generando predicción con el modelo

In [ ]:
# Escalar el número de habitaciones (RM) y la proporción industrial (INDUS)
num_rooms_std = sc_x.transform(np.array([[5.0, 10.0]]))  # Ejemplo con RM=5.0 e INDUS=10.0
price_std = slr.predict(num_rooms_std)
price = sc_y.inverse_transform(price_std)

print('El precio en miles es de %.3f' % price)

El precio en miles es de 12.865


<ipython-input-53-761dfdaebe9e>:6: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

